In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
!pip install cplex -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [3]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd

In [9]:
data = pd.read_excel('S5P3_Data.xlsx',
                     sheet_name='TSP',
                     header = 0,
                     index_col = 0,
                     usecols = 'A:F',
                     nrows = 5)
data

,City1,City2,City3,City4,City5
City1,1000000,132,217,164,58
City2,132,1000000,290,201,79
City3,217,290,1000000,113,303
City4,164,201,113,1000000,196
City5,58,79,303,196,1000000


In [7]:
data.columns

Index(['Unnamed: 0', 'City1', 'City2', 'City3', 'City4', 'City5'], dtype='object')

In [13]:
model = pyo.ConcreteModel()

model.i = pyo.Set(initialize = ['City1', 'City2', 'City3', 'City4', 'City5'])
# i = model.i
model.j = pyo.Set(initialize = ['City1', 'City2', 'City3', 'City4', 'City5'])
# j = model.j

model.ii = pyo.Set(initialize = [ 'City2', 'City3', 'City4', 'City5'])
ii = model.ii

C = data

model.x = pyo.Var(model.i, model.j, domain = pyo.Binary)
x = model.x

model.u = pyo.Var(model.i, domain = pyo.NonNegativeIntegers)
u = model.u

#Objective Function
def Objective_rule(model):
  return sum( sum( C[i][j]*x[i,j] for i in model.i) for j in model.j)

model.Obj = pyo.Objective(rule = Objective_rule, sense = pyo.minimize)

#Constraints
def Constraint1(model, j):
  return sum(x[i , j] for i in model.i ) == 1

model.Constr1 = pyo.Constraint(model.i, rule = Constraint1)

def Constraint2(model, i):
  return sum(x[i , j] for j in model.j ) == 1

model.Constr2 = pyo.Constraint(model.j, rule = Constraint2)

def Constraint3(model, i, j):
  if i != j:
    return u[i] - u[j] + 5*x[i, j] <= 4
  else:
    return u[i] - u[j] == 0

model.Constr3 = pyo.Constraint(model.ii, model.ii, rule = Constraint3)

Solver = SolverFactory('cplex_direct')


results = Solver.solve(model, tee = True)

print(results)

print("Objective Function", model.Obj())

for i in model.i:
  for j in model.j:
    print('Salesman goes from City', i, ' to City', j, '=', x[i, j]())






Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 5000000.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 1 columns.
Reduced MIP has 22 rows, 29 columns, and 86 nonzeros.
Reduced MIP has 25 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 22 rows, 29 columns, and 86 nonzeros.
Reduced MIP has 25 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Clique table members: 16.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  I